<a href="https://colab.research.google.com/github/aaaksenova/wsi_bach_thesis/blob/change/max_ari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# An Interpretable Approach to Lexical Semantic Change Detection with Lexical Substitution

Collecting data.

In [ ]:
# for_practics is a private key. since bitbucket repo is private, one first needs 
# to receive access to the repo, then share his public key with bitbucket, 
# and then the private key is used to authenticate
!eval ssh-agent -s && ssh-add for_practics && git clone git@bitbucket.org:nvanva/summer-wsi.git
!chmod 400 for_practics
!ssh-keyscan -t rsa bitbucket.org >> ~/.ssh/known_hosts
!eval `ssh-agent -s` && ssh-add for_practics && git clone git@bitbucket.org:nvanva/summer-wsi.git
!wget http://92.63.96.33/c/ilimdb_sentiment_final/files/static/bts-rnc-train-andalso.tar.gz
!tar xvzf /content/bts-rnc-train-andalso.tar.gz

SSH_AUTH_SOCK=/tmp/ssh-v9vrY8ikPaiR/agent.110; export SSH_AUTH_SOCK;
SSH_AGENT_PID=111; export SSH_AGENT_PID;
echo Agent pid 111;
Could not open a connection to your authentication agent.
/bin/bash: /root/.ssh/known_hosts: No such file or directory
Agent pid 119
Identity added: for_practics (romankazakov.krm@gmail.com)
Cloning into 'summer-wsi'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.
--2021-09-29 11:56:54--  http://92.63.96.33/c/ilimdb_sentiment_final/files/static/bts-rnc-train-andalso.tar.gz
Connecting to 92.63.96.33:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47247729 (45M) [application/x-tar]
Saving to: ‘bts-rnc-train-andalso.tar.gz’

bts-rnc-train-andal 100%[===================>]  45.06M  5.89MB/s    in 10s     

2021-09-29 11:57:04 (4.45 MB/s) - ‘bts-rnc-train-andalso.tar.gz’ saved [47247729/47247729]

xlm/russe_bts-rnc/train_1-limitNon

**Word sense induction (WSI)** is the problem of grouping occurrences of an ambiguous word according to the expressed sense of this word. Recently a new approach to this task was proposed, which generates possible substitutes for the ambiguous word in a particular context using neural language models, and then clusters sparse bag-of-words vectors built from these substitutes. Here we provide solution for the second part: clusterization of word usages using substitutes that were pregenerated based on the contexts.

Algotithm:


1.   Prepared substitutes for target words in certain contexts and their probabilities from `russe_bts-rnc` are used. Substitutes are words that could be used in the same context as the target word. Probabilities are probabilities of the particular substitution.  
2.   Substitutions are lemmatized and vectorized.
3.   BoW vectors are clustered using agglomerative clustering.
4. MaxARI is calculated.


Here all necessary modules are imported.


In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 10.6 MB/s 


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from collections import Counter
import os
from datetime import datetime
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
from pymorphy2 import MorphAnalyzer
from pathlib import Path
from time import time
import regex as re
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import BernoulliNB
from joblib import Memory
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.cluster import AgglomerativeClustering
import sklearn.cluster.hierarchical
from scipy.spatial.distance import cdist
import seaborn as sns
from sklearn.metrics import silhouette_score
from google.colab import drive
from tqdm.notebook import tqdm
from IPython.display import display

## Data collection



`substitutes_dump` – lists of substitutions with their probablities  
`data_name` – dataset with target words and examples (contexts of word usages)

In [ ]:
substitutes_dump = '/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/<mask><mask>-(а-также-T)-2ltr2f_topk150_fixspacesTrue.npz&/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/T-(а-также-<mask><mask>)-2ltr2f_topk150_fixspacesTrue.npz&'
data_name = '/content/summer-wsi/russe-wsi-kit/data/main/bts-rnc/train.csv'

Function `load_substs` collects substitutions, probabilities and examples to one main DataFrame.

In [ ]:
def load_substs(substs_fname, limit=None, drop_duplicates=False, 
                data_name = None, show=True):
    '''
    If more than one set of substitutes is used, this function combines them.
    '''
    if substs_fname.endswith('&'):
        split = substs_fname.strip('&').split('&')
        if show:
            print(f'Combining:', split)
        dfinps = [load_substs_(p, limit, drop_duplicates, data_name) \
                  for p in split]
        res = dfinps[0]
        nm = len(split[0].split('<mask>'))-1
        for dfinp in dfinps[1:]:
            res = res.merge(dfinp, on=['context','positions'], \
                            how='inner', suffixes=('','_y'))
            res.substs_probs = intersect_sparse(res.substs_probs, \
                                                res.substs_probs_y, \
                                                nmasks=nm, s=0.0)        
            res.drop(columns=[c for c in res.columns if c.endswith('_y')], \
                     inplace=True)
        if show:
            print('\nExamples for one meaning:')
            ex = res.loc[(res['word'] == 'балка') & \
                          (res['gold_sense_id'] == 1)].head(2)
            print('\nExample #1\nword:', tuple(ex.word)[0],
              '\ngold_sense_id:', tuple(ex.gold_sense_id)[0],
              '\ncontext:', tuple(ex.context)[0], '\nsubstitutes & probabilities:',
              tuple(ex.substs_probs)[0])
            print('\nExample #2\nword:', tuple(ex.word)[1],
              '\ngold_sense_id:', tuple(ex.gold_sense_id)[1],
              '\ncontext:', tuple(ex.context)[1], '\nsubstitutes & probabilities:',
              tuple(ex.substs_probs)[1])


            print('\nExamples for differrent menings:')
            ex1 = res.loc[(res['word'] == 'балка') & \
                          (res['gold_sense_id'] == 1)].head(1)
            ex2 = res.loc[(res['word'] == 'балка') & \
                          (res['gold_sense_id'] == 2)].head(1)
            print('\nExample #1\nword:', tuple(ex1.word)[0],
              '\ngold_sense_id:', tuple(ex1.gold_sense_id)[0],
              '\ncontext:', tuple(ex1.context)[0], '\nsubstitutes & probabilities:',
              tuple(ex1.substs_probs)[0])
            print('\nExample #2\nword:', tuple(ex2.word)[0],
              '\ngold_sense_id:', tuple(ex2.gold_sense_id)[0],
              '\ncontext:', tuple(ex2.context)[0], '\nsubstitutes & probabilities:',
              tuple(ex2.substs_probs)[0])
        return res
    elif substs_fname.endswith('+'): 
        split = substs_fname.strip('+').split('+') # комментарий
        p1 = '+'.join(split[:-1])
        s = float(split[-1]) 
        p2 = re.sub(r'((<mask>)+)(.*?)T',r'T\3\1',p1)
        if p2==p1:
            p2 =  re.sub(r'T(.*?)((<mask>)+)',r'\2\1T',p1)
        print(f'Combining {p1} and {p2}')
        if p1==p2:
            raise Exception('Cannot conver fname to symmetric one:', p1)
        dfinp1, dfinp2 = (load_substs_(p, limit, drop_duplicates, data_name) \
                          for p in (p1,p2))
        dfinp = dfinp1.merge(dfinp2, on=['context','positions'], how='inner', \
                             suffixes=('','_y'))
        dfinp.substs_probs = intersect_sparse(dfinp.substs_probs, \
                                              dfinp.substs_probs_y, \
                                              nmasks=len(substs_fname.split('<mask>'))-1, \
                                              s=s)
        dfinp.drop(columns=[c for c in dfinp.columns if c.endswith('_y')], \
                   inplace=True)
        return dfinp
    else:
        return load_substs_(substs_fname, limit, drop_duplicates, data_name)

In [ ]:
def load_substs_(substs_fname, limit=None, drop_duplicates=True, data_name = None):
    '''
    Collects substitutions, probabilities and examples to one main DataFrame.
    '''
    st = time()
    p = Path(substs_fname) 
    npz_filename_to_save = None
    # print(time()-st, 'Loading substs from ', p)

    # substitutions and probabilities
    if substs_fname.endswith('.npz'): 
        arr_dict = np.load(substs_fname, allow_pickle=True)
        # separate dictionaries for substitutions and probabilities
        ss,pp = arr_dict['substs'], arr_dict['probs'] 
        ss,pp = [list(s) for s in ss], [list(p) for p in pp]
        # creating a DataFrame
        substs_probs = pd.DataFrame({'substs':ss, 'probs':pp}) 
        substs_probs = substs_probs.apply(lambda r: [(p,s) for s,p in zip(r.substs, r.probs)], axis=1) 

    # examword usages and contexts
    p_ex = p.parent / (p.name+'.input')
    if os.path.isfile(p_ex):
        # print(time()-st,'Loading examples from ', p_ex)
        dfinp = pd.read_csv(p_ex, nrows=limit)
        dfinp['positions'] = dfinp['positions'].apply(pd.eval).apply(tuple)
        dfinp['word_at'] = dfinp.apply(lambda r: r.context[slice(*r.positions)], 
                                       axis=1)  # word_at stores wordform as it occured in text

    dfinp.positions = dfinp.positions.apply(tuple)
    # adding substitutions and probabilities to main DF
    dfinp['substs_probs'] = substs_probs 
    if drop_duplicates: # deleting duplicates
        dfinp = dfinp.drop_duplicates('context')
    dfinp.reset_index(inplace = True)
    # display(dfinp)
    dfinp['positions'] = dfinp.positions.apply(tuple)
    return dfinp

Here we choose best substitutes for target word using substitutes for both templates by multiplication of probabilities. 

\begin{align}
\mathcal{P}^{final} = P(s_i|Context, templ_1) \cdot P(s_i|Context, templ_2)
\end{align} 

But we do not want to throw immediately the substitute if it is in only one set (generated for one template), so we use product of smoothed probability distributions as a final proba.  

---
Here we find smoothing rates.

$\alpha_{1,2}$ is to represent the proba of any substitution not inclided in the substitutes candidate set. This evenly (across all the possible $250000$ tokens) distributes the remainder calcilated as $1 - \text{sum of all the known substitutes probas}$.

\begin{align}
\alpha_1 = \frac{1 - \sum_{i=1}^{n_{substs\_probs_1}}P_{1i}}{ 250000^{n_{masks}}}
\end{align} 

\begin{align}
\alpha_2 = \frac{1 - \sum_{i=1}^{n_{substs\_probs_2}}P_{2i}}{ 250000^{n_{masks}}}
\end{align} 

---

Then we find a smoothed probability for each substitute provided by context.

\begin{align}
P(s_i|Context, templ_1) = 
    \left\{
\begin{array}{cl}
P_{1i} + \alpha_1 & s_i \in substs\_probs_1 \\
\alpha_1 & s_i \notin substs\_probs_1
\end{array}
\right.
\end{align}  

\begin{align}
P(s_i|Context, templ_2) = 
    \left\{
\begin{array}{cl}
P_{2i} + \alpha_2 & s_i \in substs\_probs_2 \\
\alpha_2 & s_i \notin substs\_probs_2
\end{array}
\right.
\end{align}

Product of smoothed probability distributions:

\begin{align}
\mathcal{P}^{final}_{i} = 
    \left\{
\begin{array}{cl}
(P_{1i} + \alpha_1) \cdot (P2i + alpha2) & s_i \in substs\_probs_1 \land s_i \in substs\_probs_2 \\
(P_{1i} + \alpha_1) \cdot alpha2 & s_i \in substs\_probs_1 \land s_i \notin substs\_probs_2 \\
\alpha_1 \cdot (P_{2i} + \alpha_2) & s_i \notin substs\_probs_1 \land s_i \in substs\_probs_2 \\
\alpha_1 \cdot \alpha_2 & s_i \notin substs\_probs_1 \land s_i \notin substs\_probs_2
\end{array}
\right.
\end{align} 

We get two matricies (for two templates) with a probability for the substitute if it is in the certain set or zero if it is not.

\begin{align}
f_{1i} = 
    \left\{
\begin{array}{cl}
P_{1i} & s_i \in substs\_probs_1 \\
0 & s_i \notin substs\_probs_1
\end{array}
\right.
\end{align} 



\begin{align}
f_{2i} = 
    \left\{
\begin{array}{cl}
P_{2i} & s_i \in substs\_probs_2 \\
0 & s_i \notin substs\_probs_2
\end{array}
\right.
\end{align} 

Multiplication of the respective matricies ($\forall i f_{1i} \in F_1$) using product of smoothed probability distributions.

\begin{align}
\mathcal{P}^{final} = F_1 \cdot F_2 + \alpha_1 \cdot F_2 + \alpha_2 \cdot F_1 + \alpha_1 \cdot \alpha_2
\end{align} 


We do not use $\alpha_1 \cdot \alpha_2$ **in code**, because probabilities for the substitutes that are not in both sets are too low, and we need to save memory resources.

Owing to the size of the final matrix we use the sparse matrix.  
Then we find substitutes with the highest probabilities.

In [ ]:
def intersect_sparse(substs_probs, substs_probs_y, nmasks=1, s=0, debug=False):
    '''
    Combines different sets of substitutes (for different tamplates) using
    product of smoothed probability distributions.
    '''
    
    vec = DictVectorizer(sparse=True)
    f1=substs_probs.apply(lambda l: {s:p for p,s in l})
    f2=substs_probs_y.apply(lambda l: {s:p for p,s in l})
    vec.fit(list(f1)+list(f2))
    f1,f2 = (vec.transform(list(f)) for f in (f1,f2)) # sparse matrix

    alpha1, alpha2 = ((1. - f.sum(axis=-1).reshape(-1,1)) / 250000**nmasks \
                      for f in (f1, f2))
    prod = f1.multiply(f2) + f1.multiply(alpha2) + f2.multiply(alpha1)
    # + alpha1*alpha2 is ignored to preserve sparsity 
    # finally, we don't want substs with 0 
    # probs before smoothing in both distribs
    fn = np.array(vec.feature_names_)
    maxlen=(substs_probs_y.apply(len)+substs_probs.apply(len)).max()
    m = prod
    n_texts = m.shape[0]
    
    def reverse_argsort(mdata):
        return np.argsort(mdata)[::-1]
    
    idx = list()
    for text_ix in range(n_texts):
      # sparce matrices are used to preserve high performance
      # refer to https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
      # to learn the sparse matrices indexing (i.e. what is `indptr` and `data`)
      text_sparse_indices = m.indices[m.indptr[text_ix]:m.indptr[text_ix+1]]
      text_sparse_data = m.data[m.indptr[text_ix]:m.indptr[text_ix+1]]
      text_sp_data_revsorted_ixes = reverse_argsort(text_sparse_data)
      smth = text_sparse_indices[text_sp_data_revsorted_ixes]
      idx.append(smth)

    l = list()
    for text_ix, text_sparse_ixes_sorted in enumerate(idx):
      probas = m[text_ix].toarray()[0,text_sparse_ixes_sorted]
      substs = fn[text_sparse_ixes_sorted]
      good_substs = list()
      for proba, subst in zip(probas, substs):
      
        if subst.startswith(' ') and ' ' not in subst.strip():
          good_substs.append((proba, subst))
      l.append(good_substs)
 
    print('Combination: ', l[0][:10])
    return l


---

In [ ]:
substs1 = '/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/<mask><mask>-(а-также-T)-2ltr2f_topk150_fixspacesTrue.npz'
df1 = load_substs(substs1, data_name=data_name)

substs2 = '/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/T-(а-также-<mask><mask>)-2ltr2f_topk150_fixspacesTrue.npz'
df2 = load_substs(substs2, data_name=data_name)

substs3 = '/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/<mask><mask>-(а-также-T)-2ltr2f_topk150_fixspacesTrue.npz&/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/T-(а-также-<mask><mask>)-2ltr2f_topk150_fixspacesTrue.npz&'
df3 = load_substs(substs3, data_name=data_name, show=False)

Combination:  [(0.0003074132201598033, ' решетку'), (4.1475361323760285e-05, ' поверхность'), (3.976416282017758e-05, ' опору'), (2.2270818190058624e-05, ' трубу'), (2.102492416240878e-05, ' мебель'), (9.649139098880059e-06, ' спинку'), (9.437091495716981e-06, ' балку'), (8.321624868184736e-06, ' доску'), (8.03497520660322e-06, ' раму'), (7.317509182332922e-06, ' панель')]


As you can see below, combination of substitutes improves their quality and they fit context better.

In [ ]:
i = 555
print('word:', df1.loc[i, 'word'])
print('context:', df1.loc[i, 'context'])
print('<mask><mask> (а также T) - substitutes:', 
      [sub[1] for sub in df1.loc[i, 'substs_probs']][:7])
print('T (а также <mask><mask>) - substitutes:',
      [sub[1] for sub in df2.loc[i, 'substs_probs']][:7])
print('both templates - substitutes:',
      [sub[1] for sub in df3.loc[i, 'substs_probs']][:7])

word: жаба
context: , не заметил, не прочел. Грязнуха-ревность, подпрыгивая с утра до ночи, как жаба, превращала его в безумца. Однажды ночью, впрочем, открыл, когда выпиты были все
<mask><mask> (а также T) - substitutes: [' птица', ' собака', ' сова', ' рыба', ' козел', ' волчица', ' комар']
T (а также <mask><mask>) - substitutes: [' алкоголизм', ' курица', ' как ветер', ' рыба', ' сигареты', ' наркотики', ' нервы']
both templates - substitutes: [' собака', ' рыба', ' птица', ' курица', ' козел', ' волк', ' лошадь']


Here we see that substitutes for one meaning are similar and for different meanings are more different.


In [ ]:
df = load_substs(substitutes_dump, data_name=data_name)

Combining: ['/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/<mask><mask>-(а-также-T)-2ltr2f_topk150_fixspacesTrue.npz', '/content/xlm/russe_bts-rnc/train_1-limitNone-maxexperwordNone/modelNone-beamsearchFalse/T-(а-также-<mask><mask>)-2ltr2f_topk150_fixspacesTrue.npz']
Combination:  [(0.0003074132201598033, ' решетку'), (4.1475361323760285e-05, ' поверхность'), (3.976416282017758e-05, ' опору'), (2.2270818190058624e-05, ' трубу'), (2.102492416240878e-05, ' мебель'), (9.649139098880059e-06, ' спинку'), (9.437091495716981e-06, ' балку'), (8.321624868184736e-06, ' доску'), (8.03497520660322e-06, ' раму'), (7.317509182332922e-06, ' панель')]

Examples for one meaning:

Example #1
word: балка 
gold_sense_id: 1 
context: маленькой комнаты. Он был очень высок, наклонил голову, словно подпирая плечом потолочную балку, посмотрел на Сьянову серьезными черными глазами. -- Я из Москвы. Буду испытывать здесь 
substitutes & probabilities: [(0.00030741322015980

---

In [ ]:
# we will further work with this dataframe 
# (especially w the columns word, substs_probs, gold_sense_id)
df.head(3)

,index,context_id,word,gold_sense_id,predict_sense_id,positions,context,word_at,substs_probs
0,0,1,балка,1,NaN,"(90, 95)","маленькой комнаты. Он был очень высок, наклони...",балку,"[(0.0003074132201598033, решетку), (4.1475361..."
1,1,2,балка,1,NaN,"(69, 74)",Пантюхин в Склифе сейчас. Он выползти на улицу...,балка,"[(0.020003307469640956, крыша), (0.0001694636..."
2,2,3,балка,1,NaN,"(115, 122)",равнозначно обеспечивает и меланхоличную езду....,балками,"[(0.01107819609150758, колесами), (0.00561495..."
3,3,4,балка,1,NaN,"(85, 91)","верхняя часть закрыта, замкнута, многократно о...",балкой,"[(0.0018431491390275595, крышей), (0.00144177..."
4,4,5,балка,1,NaN,"(66, 71)","по телевизору: наши гол забили, я вскочил от р...",балку,"[(0.0040539591647571534, стену), (0.000310801..."
...,...,...,...,...,...,...,...,...,...
3486,3486,3487,штамп,4,NaN,"(81, 87)","дурак, но партию свою отрабатывал точно, по ус...",штампы,"[(0.0031943255190048274, партия), (9.88932979..."
3487,3487,3488,штамп,4,NaN,"(85, 91)","дело... получается, мыслить и выражать свое мн...",штампу,"[(1.0800298370078835e-05, логике), (9.5142067..."
3488,3488,3489,штамп,4,NaN,"(71, 78)",", что актер должен иметь пять штампов-клише ""п...",штампов,"[(0.0005675281293156448, линий), (0.000390070..."
3489,3489,3490,штамп,4,NaN,"(107, 113)",сегодняшний день в системе негосударственного ...,штампы,"[(0.01981654824764262, следы), (0.00222401351..."


## Preprocessing

This objects are used for morphological analysis.

In [ ]:
_ma = MorphAnalyzer()
_ma_cache = {}

Function `ma(s)` gets a string with one token, deletes spaces before and after token and returns grammatical information about it. If it was met before, we would get information from the special dictionary `_ma_cache`; if it was not, information would be gotten from `pymorphy2`.

In [ ]:
# ma stands for morphological analysis
def ma(s):
    '''
    Gets a string with one token, deletes spaces before and 
    after token and returns grammatical information about it. If it was met 
    before, we would get information from the special dictionary _ma_cache; 
    if it was not, information would be gotten from pymorphy2.
    '''
    s = s.strip()  # get rid of spaces before and after token, 
                   # pytmorphy2 doesn't work with them correctly
    if s not in _ma_cache:
        _ma_cache[s] = _ma.parse(s)
    return _ma_cache[s]

In [ ]:
print('word:', df.loc[0, 'substs_probs'][0][1])

word:  решетку


In [ ]:
print('morphological analysis:', ma(df.loc[0, 'substs_probs'][0][1]))

morphological analysis: [Parse(word='решётку', tag=OpencorporaTag('NOUN,inan,femn sing,accs'), normal_form='решётка', score=1.0, methods_stack=((DictionaryAnalyzer(), 'решётку', 8, 3),))]


Function `get_nf_cnt(substs_probs)` gets substitutes and returns normal forms of substitutes and count of substitutes that coresponds to each normal form. 

In [ ]:
def get_nf_cnt(substs_probs):
    '''
    Gets substitutes and returns normal 
    forms of substitutes and count of substitutes that coresponds to 
    each normal form.
    '''
    nf_cnt = Counter(nf for l in substs_probs \
                     for p, s in l for nf in {h.normal_form for h in ma(s)})
    print('\n'.join('%s: %d' % p for p in nf_cnt.most_common(10)))
    return nf_cnt

In [ ]:
nf_cnt = get_nf_cnt(df['substs_probs'])

камень: 1628
жена: 1124
песок: 1122
спина: 1084
ручка: 1073
кисть: 1041
стена: 1006
угол: 1005
крыло: 962
прочий: 955


Function `get_normal_forms(s, nf_cnt=None)` gets string with one token and returns set of most possible lemmas, all lemmas or one possible lemma.

In [ ]:
def get_normal_forms(s, nf_cnt=None):
    '''
    Gets string with one token and returns set of most possible lemmas, 
    all lemmas or one possible lemma.
    '''
    hh = ma(s)
    if nf_cnt is not None and len(hh) > 1:  # select most common normal form
        h_weights = [nf_cnt[h.normal_form] for h in hh]
        max_weight = max(h_weights)
        return {h.normal_form for i, h in enumerate(hh) \
                if h_weights[i] == max_weight}
    else:
        return {h.normal_form for h in hh}

For each substitute we choose the most common lemma among all substitutes' lemmas.

In [ ]:
get_normal_forms('жала')

{'жало', 'жать'}

In [ ]:
print('counts:', nf_cnt['жало'], nf_cnt['жать'])

counts: 20 21


In [ ]:
get_normal_forms('жала', nf_cnt)

{'жать'}

Function `preprocess_substs(r, lemmatize=True, nf_cnt=None, exclude_lemmas={}, )` is for preprocessing of substitutions. It gets Series of substitutions and probabilities, exclude lemmas from `exclude_lemmas` if it is not empty and lemmatize them if it is needed.

In [ ]:
def preprocess_substs(r, lemmatize=True, nf_cnt=None, exclude_lemmas={}):
    '''
    For preprocessing of substitutions. It gets Series of substitutions 
    and probabilities, exclude lemmas from exclude_lemmas 
    if it is not empty and lemmatize them if it is needed.
    '''
    res = [s.strip() for p, s in r]
    if exclude_lemmas:
        res1 = [s for s in res 
                if not set(get_normal_forms(s)).intersection(exclude_lemmas)]
        res = res1
    if lemmatize:
        res = [nf for s in res for nf in get_normal_forms(s, nf_cnt)]
    return res

Here substitutes are preprocessed for clusterization.

In [ ]:
topk = 128 # size of the substitutes' top to use

In [ ]:
substs_texts = df.apply(lambda r: preprocess_substs(r.substs_probs[:topk], 
                                                    nf_cnt=nf_cnt, 
                                                    lemmatize=True), 
                        axis=1).str.join(' ')

In [ ]:
tokens_5 = [a[1] for a in df.loc[0, 'substs_probs']]
print(' '.join(tokens_5))

 решетку  поверхность  опору  трубу  мебель  спинку  балку  доску  раму  панель  коробку  щель  палку  плотину  стул  кровать  ручку  дорожку  сетку  ширму  кисть  жену  бегу  сидение  корочку  маску  плиту  стенку  планку  стены  ободу  плечо  нишу  педаль  линию  плату  полку  лампу  крышу  столовую  кресло  розу  стонку  вилку  крышку  колонну  обои  полочку  пробку  подставку  кровлю  накладку  плоть  столик  проставку  порог  дерево  петлю  стекло  настил  стекла  арку  табличку  полосу  шахту  подушки  жижу  дугу  ковер  клетку  шею  угол  шторку  кровати  ступеньки  крошку  камни  ногами  пластину  пленку  шторы  грелку  резину  колено  грань  чашу  половую  матрац  горло  здание  проводку  заставку  плитку  диванчик  свечу  основу  матку  столбу  гладь  разводку  крыло  пятку  капель  шкафы  цедру  кухню  ведро  стенами  вышку  поясницу  вежу  вставку  шайбу  спиной  звезду  балконную  утку  бревно  спираль  печку  входную  череду  точку  мыло  шкуру  стальную  одеяла  панду  г

In [ ]:
print('word:', df.loc[0, 'word'], 
      '\nlemmas of substitutes:', substs_texts[0])

word: балка 
lemmas of substitutes: решётка поверхность опора труба мебель спинка балка доска рама панель коробка щель палка плотина стул кровать ручка дорожка сетка ширма кисть жена бег сидение корочка маска плита стенка планка стена обод плечо ниша педаль линия плата полк лампа крыша столовый кресло роза стонка вилка вилок крышка колонна обои полочка пробка подставка кровля накладка плоть столик проставка порог дерево петля стекло настил стекло арка табличка полоса шахта подушка жижа дуга ковёр клетка шея угол шторка кровать ступенька крошка камень нога пластина плёнка штора грелка резина колено грань чаша половый половой матрац горло здание проводка заставка плитка диванчик свеча основа матка столб гладь разводка крыло пятка капля шкаф цедра кухня ведро стена вышка поясница вежа вставка шайба спина звезда балконный утка уток бревно спираль печка входной черёд череда точка мыло шкура стальной одеяло панда


`substs_texts` is the set with strings of lemmatized substitutes for each example.

In [ ]:
substs_texts

0       решётка поверхность опора труба мебель спинка ...
1       крыша плита здание труба стена горка стекло до...
2       колесо диск тормоз опора мост ось колено панел...
3       крыша опора стена рама труба плита основа двер...
4       стена дерево стекло крыша камень камень кресло...
                              ...                        
3486    партия образец приём бумага график инструмент ...
3487    логика убеждение подпись нерв шаблон правило в...
3488    линия клёш картинка исключение атрибут фраза в...
3489    след остаток фрагмент корень образец стереотип...
3490    шутка сцена мотив приключение приём момент чер...
Length: 3491, dtype: object

## Clusterization and ARI calculating

Paramters which are used for vectorization of substitutes and clusterization.

In [ ]:
vectorizer = 'TfidfVectorizer'
lemmatize = True
analyzer = 'word'
min_df = 0.05
max_df = 0.95
ngram_range = (1, 1)
ncs=(2,10)

In [ ]:
vec = eval(vectorizer)(token_pattern=r"(?u)\b\w+\b", 
                       min_df=min_df, max_df=max_df, 
                       analyzer=analyzer, ngram_range=ngram_range)

In [ ]:
# the list of words of interest. 
# note that all the dataset entries refer to one of these words
df.word.unique()

array(['балка', 'вид', 'винт', 'горн', 'губа', 'жаба', 'клетка', 'крыло',
       'купюра', 'курица', 'лавка', 'лайка', 'лев', 'лира', 'мина',
       'мишень', 'обед', 'оклад', 'опушка', 'полис', 'пост', 'поток',
       'проказа', 'пропасть', 'проспект', 'пытка', 'рысь', 'среда',
       'хвост', 'штамп'], dtype=object)

Function `max_ari` gets data and substitutions (and some parameters like `vectorizer` and parameters for clusterization). Here for each unique word substitutions are vectorized and senses are clusterized. 

In [ ]:
def max_ari(df, X, ncs,
            affinity='cosine', linkage='average', vectorizer=None):
    '''
    Gets data and substitutions (and some parameters 
    like vectorizer and parameters for clusterization). 
    Here for each unique word substitutions 
    are vectorized and senses are clusterized.
    It returns metrics of clusterization.
    '''
    sdfs = []
    for word in df.word.unique():
        # collecting examples for the word
        mask = (df.word == word)
        # vectors for substitutions
        vectors = X[mask] if vectorizer is None \
        else vectorizer.fit_transform(X[mask]).toarray()
        # ids of senses of the examples
        gold_sense_ids = df.gold_sense_id[mask]
        gold_sense_ids = None if gold_sense_ids.isnull().any() \
        else gold_sense_ids

        # clusterization (ari is kept in sdf along with other info)
        best_clids, sdf, _ = clusterize_search(word, vectors, gold_sense_ids, 
                ncs=ncs,
                affinity=affinity, linkage=linkage)
        df.loc[mask, 'predict_sense_id'] = best_clids # result ids of clusters
        sdfs.append(sdf)
    
    return sdfs

Function `clusterize_search` gets word, vectors, gold_sense_ids and provides AgglomerativeClustering of vectors of substitutes for each example of one word.


Than it calculates [ARI](https://en.wikipedia.org/wiki/Rand_index) and [silhouette scores](https://en.wikipedia.org/wiki/Silhouette_(clustering)).



In [ ]:
def clusterize_search(word, vecs, gold_sense_ids = None, 
                      ncs=list(range(1, 5, 1)) + list(range(5, 12, 2)),
            affinity='cosine', linkage='average', print_topf=None,
            generate_pictures_df = False,  corpora_ids = None):
    '''
    Gets word, vectors, gold_sense_ids and provides AgglomerativeClustering.
    '''
  
    sdfs = []
    tmp_dfs = []

    # adding 1 to zero vectors, because there will be problems (all-zero vectorized entries), if they remain zero
    # this introduces a new dimension with possible values 
    # 1 -- all the other coords are zeros
    # 0 -- other coords are not all-zeros
    zero_vecs = ((vecs ** 2).sum(axis=-1) == 0)
    if zero_vecs.sum() > 0:
        vecs = np.concatenate((vecs, 
                               zero_vecs[:, np.newaxis].astype(vecs.dtype)), 
                              axis=-1)

    best_clids = None
    best_silhouette = 0
    distances = []

    # matrix with computed distances between each pair of the two collections of inputs
    distance_matrix = cdist(vecs, vecs, metric=affinity)
    distances.append(distance_matrix)

    for nc in ncs:
        # clusterization
        clr = AgglomerativeClustering(affinity='precomputed', 
                                      linkage=linkage, 
                                      n_clusters=nc)
        clids = clr.fit_predict(distance_matrix) if nc > 1 \
        else np.zeros(len(vecs))

        # computing metrics
        ari = ARI(gold_sense_ids, clids) if gold_sense_ids is not None else np.nan
        sil_cosine = -1. if len(np.unique(clids)) < 2 
                     else silhouette_score(vecs, clids, metric='cosine')
        sil_euclidean = -1. if len(np.unique(clids)) < 2 
                        else silhouette_score(vecs, clids, metric='euclidean')
        
        # vc like 5/4/3 says that 
        # there are 5 examples w golden_id=id1; 
        # there are 4 examples w golden_id=id2; 
        # there are 3 examples w golden_id=id3; 
        # e.g. вид 4/3/2 means that there were 
        # 4 examples with вид==view; 3 examples .==type; 2 examples .==specie
        vc = '' if gold_sense_ids is None else '/'.join(
            np.sort(pd.value_counts(gold_sense_ids).values)[::-1].astype(str))
        
        if sil_cosine > best_silhouette:
            best_silhouette = sil_cosine
            best_clids = clids

        # metrics for each word
        sdf = pd.DataFrame({'ari': ari,
                            'word': word, 'nc': nc,
                            'sil_cosine': sil_cosine,
                            'sil_euclidean': sil_euclidean,
                            'vc': vc,
                            'affinity': affinity, 'linkage': linkage}, \
                           index=[0])

        sdfs.append(sdf)
  
    sdf = pd.concat(sdfs, ignore_index=True)

    return best_clids, sdf, distances


In [ ]:
sdfs = max_ari(df, 
               substs_texts, 
               ncs=range(*ncs), 
               affinity='cosine', 
               linkage='average', 
               vectorizer=vec)

Metrics for one unique word.

In [ ]:
sdfs[0]

,ari,word,nc,sil_cosine,sil_euclidean,vc,affinity,linkage
0,0.834313,балка,2,0.149469,0.079886,81/38,cosine,average
1,0.753423,балка,3,0.119797,0.064259,81/38,cosine,average
2,0.733763,балка,4,0.102718,0.055423,81/38,cosine,average
3,0.492503,балка,5,0.112496,0.060376,81/38,cosine,average
4,0.410899,балка,6,0.114486,0.061526,81/38,cosine,average
5,0.388491,балка,7,0.095982,0.051460,81/38,cosine,average
6,0.280409,балка,8,0.110435,0.059147,81/38,cosine,average
7,0.271157,балка,9,0.115767,0.061779,81/38,cosine,average


In [ ]:
def metrics(sdfs):
    # all metrics for each unique word
    sdf = pd.concat(sdfs, ignore_index=True)
    # groupby is docuented to preserve inside group order
    res = sdf.sort_values(by='ari').groupby(by='word').last()
    # maxari for fixed hypers
    fixed_hypers = sdf.groupby(['affinity', 
                                'linkage', 
                                'nc']).agg({'ari': np.mean}).reset_index()
    idxmax = fixed_hypers.ari.idxmax()
    res_df = fixed_hypers.loc[idxmax:idxmax].copy()
    res_df = res_df.rename(columns=lambda c: 'fh_maxari' if c == 'ari' \
                           else 'fh_' + c)
    res_df['maxari'] = res.ari.mean()

    for metric in [c for c in sdf.columns if c.startswith('sil')]:
        res_df[metric+'_ari'] = sdf.sort_values(by=metric).groupby(by='word').last().ari.mean()

    return res_df, res, sdf

In [ ]:
res_df, res, sdf = metrics(sdfs)

## Results

General metrics for the configuration.

In [ ]:
res_df

,fh_affinity,fh_linkage,fh_nc,fh_maxari,maxari,sil_cosine_ari,sil_euclidean_ari
1,cosine,average,3,0.563798,0.685901,0.517858,0.517858


Metrics for each word.

In [ ]:
res

,ari,nc,sil_cosine,sil_euclidean,vc,affinity,linkage
word,,,,,,,
балка,0.834313,2,0.149469,0.079886,81/38,cosine,average
вид,0.774342,2,0.094609,0.050402,38/36/3,cosine,average
винт,0.314024,7,0.130702,0.071634,67/39/16/1,cosine,average
горн,0.617461,2,0.099717,0.053190,30/20/1,cosine,average
губа,0.738749,2,0.213769,0.117009,132/3/2,cosine,average
жаба,0.507929,5,0.162723,0.089471,79/27/9/6,cosine,average
клетка,0.799571,6,0.126598,0.068017,96/38/7/4/4/1,cosine,average
крыло,0.562013,4,0.122369,0.067302,51/19/7/5/4/3/1/1,cosine,average
купюра,0.899256,3,0.113443,0.061212,138/12,cosine,average
